In [38]:
%store -r normalized_df

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [47]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils

#X = normalized_df[['Sympt_blødning', 'Sympt_smerter', 'Sympt_ascites', 'Sympt_fatigue']]
#X = normalized_df.drop(['kreftform'], axis=1)
X = normalized_df[['Sympt_blødning', 'Sympt_smerter', 'Sympt_ascites', 'Sympt_fatigue', 'Lengde_sympt_dager', 'Lengde_sympt_uker', 'Lengde_sympt_mnd']]
y = normalized_df[['kreftform']]

def run_once(graph):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

    lab_enc = preprocessing.LabelEncoder()
    encoded_y_train = lab_enc.fit_transform(y_train.values.ravel())
    encoded_y_test = lab_enc.fit_transform(y_test.values.ravel())
    
    clf = RandomForestClassifier(n_estimators=100)

    clf.fit(X_train, encoded_y_train)
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(encoded_y_test, y_pred)
    print(confusion_matrix(encoded_y_test, y_pred))
    print(accuracy_score(encoded_y_test, y_pred))
    
    if(graph == True):
        estimator = clf.estimators_[5]

        from sklearn.tree import export_graphviz
        # Export as dot file
        export_graphviz(estimator, out_file='tree.dot',
                        feature_names = X.columns.values,
                        class_names = ['corpus', 'ovarie' , 'cervix'],
                        rounded = True, proportion = False, 
                        precision = 2, filled = True)

        # Convert to png using system command (requires Graphviz)
        from subprocess import call
        call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

        # Display in jupyter notebook
        from IPython.display import Image
        Image(filename = 'tree.png')
    
def run_more_than_once(run_number):
    tot_acc = 0
    max_matrix = 0
    max_accuracy = 0

    min_matrix = 0
    min_accuracy = 1
    for i in range(run_number):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

        lab_enc = preprocessing.LabelEncoder()
        encoded_y_train = lab_enc.fit_transform(y_train.values.ravel())
        encoded_y_test = lab_enc.fit_transform(y_test.values.ravel())
        
        clf = RandomForestClassifier(n_estimators=100)

        clf.fit(X_train, encoded_y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(encoded_y_test, y_pred)

        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_matrix = confusion_matrix(encoded_y_test, y_pred)

        if accuracy < min_accuracy:
            min_accuracy = accuracy
            min_matrix = confusion_matrix(encoded_y_test, y_pred)

        tot_acc += accuracy
        tot = i+1
        mean = tot_acc/tot

    print('Maximum')
    print(max_accuracy, '\n', max_matrix)
    print('----------------------------------')
    print('Minimum')
    print(min_accuracy, '\n', min_matrix)
    print('----------------------------------')
    print('mean accuracy:', mean)

In [48]:
run_more_than_once(100)

Maximum
0.6888888888888889 
 [[ 22  46   0]
 [  6 132   3]
 [  4  11   1]]
----------------------------------
Minimum
0.5733333333333334 
 [[ 20  56   4]
 [  9 109   3]
 [  5  19   0]]
----------------------------------
mean accuracy: 0.6315555555555554


In [35]:
run_once(False)

[[ 12  41   0]
 [  0 101   0]
 [  3  13   0]]
0.6647058823529411
